# The Big Bang Theory Exploratory Data Analysis

### Imports:

In [2]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

## Loading sample data file:

In [ ]:
# with open("sample.json", 'r') as file:
#     sampleData = json.load(file)

In [3]:
with open("DT_2/Raw/S1/The Big Bang_S0104.json", 'r') as file:
    sampleData = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'DT_2/Raw/S1/The Big Bang_S0104.json'

In [27]:
sampleData["Dialog 3"]

{'Scene': 'Sheldon and Leonard’s apartment.',
 'Participant': ['Sheldon', 'Leonard'],
 'GT': 1,
 'AV_ID': 'The Big Bang_S0104',
 'Humor Start Time': '00:00:19',
 'Humor End Time': '00:00:20',
 'Dialog Turns 0': {'Recipients': ['Leonard'],
  'Speaker': 'Sheldon',
  'Dialog': 'Put it on a backburner.',
  'Dialog Start time': '00:00:08:00',
  'Dialog End time': '00:00:09:09'},
 'Dialog Turns 1': {'Recipients': ['Leonard'],
  'Speaker': 'Sheldon',
  'Dialog': "Anyway. It occurs to me, If I have ever did perfect time machine, I'd just go into the past and give it to myself. That's eliminating the need for me to invent it in the first place.",
  'Dialog Start time': '00:00:10:09',
  'Dialog End time': '00:00:19:09'}}

In [ ]:
num_dialog_turns = sum(1 for key in sampleData[] if key.startswith('Dialog Turns'))-1

In [26]:
laughter_b4_dialog_count = 0
laughter_b4_dialog = []

# def laughterBeforeDialog(sampleData):

for dialog_num, info in sampleData.items():
    num_dialog_turns = "Dialog Turns "+ str(sum(1 for key in info if key.startswith('Dialog Turns'))-1)
    start_dialog = info["Dialog Turns 0"]["Dialog Start time"]
    end_dialog = info[num_dialog_turns]["Dialog End time"]

    try:
        laughter_start = info["Humor start time"]
        laughter_end = info["Laughter end time"]

        print(laughter_start)
        
        try:
            start_dialog_datetime = datetime.strptime(start_dialog.strip(), '%H:%M:%S:%f')

        except ValueError:
            start_dialog_datetime = datetime.strptime(start_dialog.strip(), '%H:%M:%S,%f')

        laughter_start_datetime = datetime.strptime(laughter_start, '%H:%M:%S')

        if laughter_start_datetime < start_dialog_datetime:
            print("Laughter starts BEFORE dialog")
            laughter_b4_dialog_count += 1 
            laughter_b4_dialog.append(dialog_num)
        else:
            print("Laughter starts BEFORE dialog")

    except KeyError:
        laughter_start = None
        laughter_end = None

In [13]:
# laughterBeforeDialog(sampleData42)